# Rosetta Constraint API

@Author: 吴炜坤

@email: weikun.wu@xtalpi.com

在前面的章节中，我们介绍了在Rosetta中如何定义需要的约束文件。这些文件的定义和约束都是手动的。 在本章节中，我们将会介绍:

如何从file读取constraint至pose中
如何使用constraint生成器自动生成约束
如何管理Pose中的约束

### 一、从几何约束文件读取约束

这里我们将使用ROSETTA中的constraint file（限制文件）。
举例，我们当前对20号和6号Ca原子进行距离的约束，目标约束距离为9埃。

constraint file的内容如下:
```
AtomPair CA 20 CA 6 LINEAR_PENALTY 9.0 0 0 1.0
```

In [1]:
# 在1ubq_clean.pdb的例子上施加原子对的限制

from pyrosetta import init, create_score_function, pose_from_pdb
from pyrosetta.rosetta.core.scoring import ScoreType
init()

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release 2021.26+release.b308454c455dd04f6824cc8b23e54bbb9be2cdd7 2021-07-02T13:01:54] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release r288 2021.26+release.b308454c455 b308454c455dd04f6824cc8b23e54bbb9be2cdd7 http://www.pyrosetta.org 2021-07-02T13:01:54
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-1705239655 seed_offset=0 real_seed=-1705239655 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-1705239655 RG_type=mt19937


In [2]:
# 初始化REF2015的打分函数对象:
scorefxn = create_score_function('ref2015')

# 激活相关惩罚项的打分函数权重:
scorefxn.set_weight(ScoreType.atom_pair_constraint, 1.0) 

core.scoring.etable: {0} Starting energy table calculation
core.scoring.etable: {0} smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: {0} smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: {0} smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: {0} Finished calculating energy tables.
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
core.chemical.GlobalResidueTypeSet: {0} Finished initializing fa_standard residue type set.

从文件读取约束信息主要依靠ConstraintSetMover的作用:

In [3]:
from pyrosetta.rosetta.protocols.constraint_movers import ConstraintSetMover
cst_set = ConstraintSetMover()
cst_set.add_constraints(False) # True=在原有限制基础上额外再添加限制，False= 从文件中读取并覆盖所有的限制。
cst_set.constraint_file('./data/constraint_file')

In [4]:
# load pose from 1ubq_clean.pdb
pose = pose_from_pdb("./data/1ubq_clean.pdb")
cst_set.apply(pose)

core.import_pose.import_pose: {0} File './data/1ubq_clean.pdb' automatically determined to be of type PDB
core.scoring.constraints.ConstraintsIO: {0} read constraints from ./data/constraint_file
core.scoring.constraints.ConstraintsIO: {0} Read in 1 constraints


In [5]:
# 打印查看当前pose所有设定的constraint
print(pose.constraint_set())

ResiduePairConstraints: total: 20   plotting active...
ResiduePairConstraints (6,20)
AtomPairConstraint (2,20-2,6)
         r      func     dfunc dfunc_est
         2     7.000    -1.000    -1.000
       2.5     6.500    -1.000    -1.000
         3     6.000    -1.000    -1.000
       3.5     5.500    -1.000    -1.000
         4     5.000    -1.000    -1.000
       4.5     4.500    -1.000    -1.000
         5     4.000    -1.000    -1.000
       5.5     3.500    -1.000    -1.000
         6     3.000    -1.000    -1.000
       6.5     2.500    -1.000    -1.000
         7     2.000    -1.000    -1.000
       7.5     1.500    -1.000    -1.000
         8     1.000    -1.000    -1.000
       8.5     0.500    -1.000    -1.000
         9     0.000     0.000     0.000
       9.5     0.500     1.000     1.000
        10     1.000     1.000     1.000
      10.5     1.500     1.000     1.000
        11     2.000     1.000     1.000
      11.5     2.500     1.000     1.000
        12     3.000    

从这个简单的例子中可以看到，使用“限制文件”对pose进行限制的方法理论上适用于所有的场景。因为只要我们按照Rosetta的要求编写所有的限制规则于限制文件中，这种方法就可以最大化用户自由度，但是缺点是使用“限制文件”事先需要对限制文件的编写规则有一定的了解和把握。

### 二、从序列约束文件读取约束

主要通过AddCompositionConstraintMover，AddNetChargeConstraintMover两个函数来进行约束的添加。

#### 2.1 AddCompositionConstraintMover
给氨基酸的组成比例添加限制。开始使用前，需要准备*.comp限制文件，控制如何限制氨基酸的比例。

In [6]:
from pyrosetta.rosetta.core.scoring import ScoreType
score = create_score_function('ref2015')
score.set_weight(ScoreType.aa_composition, 1.0) # reweight score

from pyrosetta.rosetta.protocols.aa_composition import AddCompositionConstraintMover
add_comp_cst = AddCompositionConstraintMover()
add_comp_cst.create_constraint_from_file('./data/aa_comp_constraint_file.comp')
# add_comp_cst.add_residue_selector(selector) # 输入pose限制的范围, 如果缺省，那么将全pose限制氨基酸组成。
add_comp_cst.apply(pose)

protocols.aa_composition.AddCompositionConstraintMover: {0} Initialized AACompositionConstraint object from file ./data/aa_comp_constraint_file.comp.


#### 2.2 AddNetChargeConstraintMover
添加净电荷限制Mover,开始使用前，需要准备.charge限制文件。

In [7]:
from pyrosetta.rosetta.core.scoring import ScoreType
score = create_score_function('ref2015')
score.set_weight(ScoreType.netcharge, 1.0) # reweight score

from pyrosetta.rosetta.protocols.aa_composition import AddNetChargeConstraintMover
add_net_charge_cst = AddNetChargeConstraintMover()
add_net_charge_cst.create_constraint_from_file('./data/charge_comp_constraint_file.charge')
# add_net_charge_cst.add_residue_selector(selector) # 输入pose限制的范围, 如果缺省，那么将全pose限制氨基酸组成。
add_net_charge_cst.apply(pose)

core.scoring.netcharge_energy.NetChargeEnergySetup: {0} Reading netcharge scoring term setup data from ./data/charge_comp_constraint_file.charge.
protocols.aa_composition.AddNetChargeConstraintMover: {0} Initialized NetChargeConstraint object from file ./data/charge_comp_constraint_file.charge.


#### 2.3 ResidueTypeConstraintMover
在Design时倾向于奖励某类氨基酸的能量得分。
如当突变成Gly时 ddg=-1，突变成A的ddg=-1，那么经过ALA bonus约束(weight=2.0). 最后氨基酸序列输出时，选择了突变成A氨基酸的ddg为-2。 这是一种直接在packer中奖励residue序列得分全局的方法。

In [8]:
from pyrosetta.rosetta.protocols.constraint_movers import ResidueTypeConstraintMover
aa_type_cst = ResidueTypeConstraintMover()
aa_type_cst.set_AA_name3('ALA')
aa_type_cst.set_favor_bonus(2.0) #权重
aa_type_cst.apply(pose)

### 三、使用Constraint Generators生成约束
Constraint Generators不同于file文件系统，生成器可以自动地根据用户输入的参数进行约束参数生成，十分方便。

以下我们将介绍几种常用的自动生成器的使用方法:

#### 3.1 AddConstraints
AddConstraintsMover可以使用多个constraint generator来生成一些列的限制，并且将他们添加到Pose中。

后面将在例子中进行使用说明。

#### 3.2 AtomPairConstraintGenerator
在一个selector子集内部生成**遍历的Pair-wise的原子距离约束**。

有两种使用模式:
- 模式1: 在一个子集中生成距离限制

In [9]:
from pyrosetta.rosetta.protocols.simple_moves import VirtualRootMover

# load pose from 1ubq_clean.pdb
pose = pose_from_pdb("./data/1ubq_clean.pdb")

# 添加虚拟的root
vr_mover = VirtualRootMover()
vr_mover.apply(pose)

core.import_pose.import_pose: {0} File './data/1ubq_clean.pdb' automatically determined to be of type PDB


In [10]:
# Score reweight
score = create_score_function('ref2015')
score.set_weight(ScoreType.atom_pair_constraint, 1.0) # reweight constraint score

# 定义生成器
from pyrosetta.rosetta.protocols.constraint_generator import AtomPairConstraintGenerator
atompair_cst = AtomPairConstraintGenerator()

# 定义残基选择器
from pyrosetta.rosetta.core.select.residue_selector import ChainSelector
chain_selector = ChainSelector(1)

# 结构设置
atompair_cst.set_residue_selector(chain_selector)

# 参数设置
atompair_cst.set_ca_only(True)
atompair_cst.set_max_distance(8) # 超出这个距离不添加限制；
atompair_cst.set_min_seq_sep(6)  # 如果序列氨基酸号大于该值才添加限制。
atompair_cst.set_sd(1.0) # distance constraint的标准误;
atompair_cst.set_use_harmonic_function(True) # use harmonic function instead of SOG function

# add AtomPairConstraintGenerator to pose;
from pyrosetta.rosetta.protocols.constraint_generator import AddConstraints
add_cst = AddConstraints()
add_cst.add_generator(atompair_cst)
add_cst.apply(pose)

protocols.constraint_generator.AddConstraints: {0} Adding 116 constraints generated by ConstraintGenerator named unnamed_constraint_generator


- 模式2: 生成两个子集之间的距离限制

In [11]:
from pyrosetta.rosetta.protocols.simple_moves import VirtualRootMover

# load pose from 1ubq_clean.pdb
pose = pose_from_pdb("./data/1ubq_clean.pdb")

# 添加虚拟的root
vr_mover = VirtualRootMover()
vr_mover.apply(pose)

core.import_pose.import_pose: {0} File './data/1ubq_clean.pdb' automatically determined to be of type PDB


In [12]:
# Score reweight
score = create_score_function('ref2015')
score.set_weight(ScoreType.atom_pair_constraint, 1.0) # reweight constraint score

# 定义生成器
from pyrosetta.rosetta.protocols.constraint_generator import AtomPairConstraintGenerator
atompair_cst = AtomPairConstraintGenerator()

# 定义残基选择器
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
res1_selector = ResidueIndexSelector('1-10')
res2_selector = ResidueIndexSelector('20-30')

# 结构设置
atompair_cst.set_residue_selector(res1_selector)
atompair_cst.set_secondary_residue_selector(res2_selector) #生成两个selector之间的距离限制时需要，使用后，min_seq_sep将不起效。

# 参数设置
atompair_cst.set_ca_only(True)
atompair_cst.set_max_distance(12) # 超出这个距离不添加限制；
atompair_cst.set_sd(1.0) # distance constraint的标准误;
atompair_cst.set_use_harmonic_function(True) # use harmonic function instead of SOG function

# add AtomPairConstraintGenerator to pose;
from pyrosetta.rosetta.protocols.constraint_generator import AddConstraints
add_cst = AddConstraints()
add_cst.add_generator(atompair_cst)
add_cst.apply(pose)

protocols.constraint_generator.AddConstraints: {0} Adding 14 constraints generated by ConstraintGenerator named unnamed_constraint_generator


#### 3.3 CoordinateConstraintGenerator
在一个Selector子集内部生成坐标限制;

注意: 需要使用pyrosetta.rosetta.protocols.simple_moves.VirtualRootMover加一个虚拟的root.

In [13]:
from pyrosetta.rosetta.protocols.simple_moves import VirtualRootMover

# load pose from 1ubq_clean.pdb
pose = pose_from_pdb("./data/1ubq_clean.pdb")

# 添加虚拟的root
vr_mover = VirtualRootMover()
vr_mover.apply(pose)

core.import_pose.import_pose: {0} File './data/1ubq_clean.pdb' automatically determined to be of type PDB


In [14]:
# Score reweight
score = create_score_function('ref2015')
score.set_weight(ScoreType.coordinate_constraint, 1.0) # reweight constraint score

# 定义残基选择器
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
res1_selector = ResidueIndexSelector('1-10')


# 定义坐标约束生成器
from pyrosetta.rosetta.protocols.constraint_generator import CoordinateConstraintGenerator
coordinate_cst = CoordinateConstraintGenerator()
coordinate_cst.set_ca_only(False)
coordinate_cst.set_reference_pose(pose)
coordinate_cst.set_residue_selector(res1_selector)
coordinate_cst.set_sidechain(False) # True = all重原子，False = 骨架重原子

# 约束函数设定
coordinate_cst.set_sd(1.0) # strength/deviation e.g. -relax:coord_cst_stdev

# add AtomPairConstraintGenerator to pose;
from pyrosetta.rosetta.protocols.constraint_generator import AddConstraints
add_cst = AddConstraints()
add_cst.add_generator(coordinate_cst)
add_cst.apply(pose)

protocols.constraint_generator.AddConstraints: {0} Adding 40 constraints generated by ConstraintGenerator named unnamed_constraint_generator


#### 3.4 DihedralConstraintGenerator
基于骨架二面角(phi or psi or omega)自动生成约束的生成器, 默认使用CircularHarmonic函数。

In [22]:
from pyrosetta.rosetta.protocols.simple_moves import VirtualRootMover

# load pose from 1ubq_clean.pdb
pose = pose_from_pdb("./data/1ubq_clean.pdb")

# 添加虚拟的root
vr_mover = VirtualRootMover()
vr_mover.apply(pose)

core.import_pose.import_pose: {0} File './data/1ubq_clean.pdb' automatically determined to be of type PDB


In [24]:
from pyrosetta.rosetta.protocols.constraint_generator import DihedralConstraintGenerator
from pyrosetta.rosetta.core.id import MainchainTorsionType

# Score reweight
score = create_score_function('ref2015')
score.set_weight(ScoreType.dihedral_constraint, 1.0) # reweight score

# 
dh_cst = DihedralConstraintGenerator()
# dh_cst.set_residue_selector()  # 设置选择器
dh_cst.set_sd_degree(7.0)  # 默认允许的最大变化角度
dh_cst.set_torsion_type(MainchainTorsionType.phi_dihedral)  # phi_dihedral,psi_dihedral,omega_dihedral

# add DihedralConstraintGenerator to pose;
from pyrosetta.rosetta.protocols.constraint_generator import AddConstraints
add_cst = AddConstraints()
add_cst.add_generator(termin_cst)
add_cst.apply(pose)

protocols.constraint_generator.TerminiConstraintGenerator: {0} Constraining atoms  atomno= 2 rsd= 1  and  atomno= 2 rsd= 76 , min_distance=8 max_distance=20
protocols.constraint_generator.AddConstraints: {0} Adding 1 constraints generated by ConstraintGenerator named unnamed_constraint_generator


#### 3.5 TerminiConstraintGenerator
生成N端和C端之间原子的限制参数。从PyRosetta API中看，仅仅是可以设置简单的距离限制。

In [15]:
from pyrosetta.rosetta.protocols.simple_moves import VirtualRootMover

# load pose from 1ubq_clean.pdb
pose = pose_from_pdb("./data/1ubq_clean.pdb")

# 添加虚拟的root
vr_mover = VirtualRootMover()
vr_mover.apply(pose)

core.import_pose.import_pose: {0} File './data/1ubq_clean.pdb' automatically determined to be of type PDB


In [16]:
# Score reweight
score = create_score_function('ref2015')
score.set_weight(ScoreType.atom_pair_constraint, 1.0) # reweight score

# 
from pyrosetta.rosetta.protocols.constraint_generator import TerminiConstraintGenerator
termin_cst = TerminiConstraintGenerator()
termin_cst.set_min_distance(8)
termin_cst.set_max_distance(20)
termin_cst.set_sd(1.0)

# add TerminiConstraintGenerator to pose;
from pyrosetta.rosetta.protocols.constraint_generator import AddConstraints
add_cst = AddConstraints()
add_cst.add_generator(termin_cst)
add_cst.apply(pose)

protocols.constraint_generator.TerminiConstraintGenerator: {0} Constraining atoms  atomno= 2 rsd= 1  and  atomno= 2 rsd= 76 , min_distance=8 max_distance=20
protocols.constraint_generator.AddConstraints: {0} Adding 1 constraints generated by ConstraintGenerator named unnamed_constraint_generator


#### 3.6 AutomaticSheetConstraintGenerator
自动根据Pose中的beta sheet结构匹配，生成sheet间氢键的配对约束。

In [17]:
from pyrosetta.rosetta.protocols.fold_from_loops.constraint_generator import AutomaticSheetConstraintGenerator
# Score reweight
score = create_score_function('ref2015')
score.set_weight(ScoreType.atom_pair_constraint, 1.0) # reweight score

sheet_g = AutomaticSheetConstraintGenerator()
sheet_g.distance(12)
sheet_g.sd(1.0)

# add AutomaticSheetConstraintGenerator to pose;
from pyrosetta.rosetta.protocols.constraint_generator import AddConstraints
add_cst = AddConstraints()
add_cst.add_generator(termin_cst)
add_cst.apply(pose)

protocols.constraint_generator.TerminiConstraintGenerator: {0} Constraining atoms  atomno= 2 rsd= 1  and  atomno= 2 rsd= 76 , min_distance=8 max_distance=20
protocols.constraint_generator.AddConstraints: {0} Adding 1 constraints generated by ConstraintGenerator named unnamed_constraint_generator


### 四、清除Pose中的约束限制的方法

#### 4.1 ReleaseConstraintFromResidueMover
从Pose中清除selector中的氨基酸限制。（ResidueSelector介绍详见第五章节）

In [18]:
from pyrosetta.rosetta.protocols.fold_from_loops.movers import ReleaseConstraintFromResidueMover
from pyrosetta.rosetta.core.select.residue_selector import ChainSelector

# 定义残基选择器
chain_selector = ChainSelector(1)

# 清除约束
clean_selector_cst = ReleaseConstraintFromResidueMover(chain_selector)
clean_selector_cst.apply(pose)

#### 4.2 RemoveConstraints
从Pose中清除由constraint generator生成的限制。

In [19]:
# add TerminiConstraintGenerator to pose again;
from pyrosetta.rosetta.protocols.constraint_generator import AddConstraints
add_cst = AddConstraints()
add_cst.add_generator(termin_cst)
add_cst.apply(pose)


from pyrosetta.rosetta.protocols.constraint_generator import RemoveConstraints
rm_cst = RemoveConstraints()
rm_cst.add_generator(termin_cst)  # termin_cst为之前定义的生成器;
rm_cst.apply(pose)

protocols.constraint_generator.TerminiConstraintGenerator: {0} Constraining atoms  atomno= 2 rsd= 1  and  atomno= 2 rsd= 76 , min_distance=8 max_distance=20
protocols.constraint_generator.AddConstraints: {0} Adding 1 constraints generated by ConstraintGenerator named unnamed_constraint_generator
protocols.constraint_generator.RemoveConstraints: {0} Removing 1 constraints from pose generated by unnamed_constraint_generator


#### 4.3 ClearConstraintsMover
从Pose中清除所有的限制。

In [20]:
from pyrosetta.rosetta.protocols.constraint_movers import ClearConstraintsMover
clear_cst = ClearConstraintsMover()
clear_cst.apply(pose)

#### 4.4 ClearCompositionConstraintsMover
从Pose中清除sequence constraint/net charge constraint.

In [21]:
from pyrosetta.rosetta.protocols.aa_composition import ClearCompositionConstraintsMover
clear_seq_cst = ClearCompositionConstraintsMover()
clear_seq_cst.apply(pose)

protocols.aa_composition.ClearCompositionConstraintsMover: {0} Removed all sequence constraints from the pose.
